In [ ]:
!git clone https://github.com/clementgr/stain-normalization.git

Cloning into 'stain-normalization'...
remote: Enumerating objects: 77, done.
remote: Counting objects: 100% (77/77), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 77 (delta 37), reused 59 (delta 22), pack-reused 0
Unpacking objects: 100% (77/77), done.


In [ ]:
import os
os.chdir('stain-normalization')

In [ ]:
!sh scripts/data_download.sh

Downloading...
From: https://drive.google.com/uc?id=1MKbYEpXvLScIGvpZMDF-6ZWwx66lap6J
To: /content/stain-normalization/extracted_patches.zip
1.33GB [00:10, 128MB/s]


In [ ]:
%%capture
!unzip extracted_patches.zip
!rm extracted_patches.zip

split trainA, trainB, valA, valB, testA, testB (actually only need train & test)

In [ ]:
import os
import random
import shutil
from sklearn.model_selection import train_test_split
random.seed(21)

In [ ]:
root_dir = 'extracted_patches'
aperio_slides = [s for s in os.listdir(root_dir) if s[0] == 'A']
hamamatsu_slides = [s for s in os.listdir(root_dir) if s[0] == 'H']
print(len(aperio_slides), '|', len(hamamatsu_slides))

152 | 152


In [ ]:
enable_val = False
test_size = 0.2
if enable_val:
  test_size = 0.3
  val_size = 0.5

In [ ]:
train, val, test = [], [], []
for s in aperio_slides:
  slide_dir = os.path.join(root_dir, s)
  imgs = [os.path.join(root_dir, s, i) for i in os.listdir(slide_dir)]
  random.shuffle(imgs)
  subtrain, subtest = train_test_split(imgs, test_size=test_size)
  if enable_val:
    subval, subtest = train_test_split(subtest, test_size=val_size)
    val.extend(subval)
  train.extend(subtrain)
  test.extend(subtest)

In [ ]:
os.makedirs('data/trainA')
os.makedirs('data/testA')
if enable_val:
  os.makedirs('data/valA')

In [ ]:
for src in train:
  s = src.split('/')[-2]
  img = src.split('/')[-1]
  dst = f'data/trainA/{s}_{img}'
  shutil.copy(src, dst)

for src in test:
  s = src.split('/')[-2]
  img = src.split('/')[-1]
  dst = f'data/testA/{s}_{img}'
  shutil.copy(src, dst)

if enable_val:
  for src in val:
    s = src.split('/')[-2]
    img = src.split('/')[-1]
    dst = f'data/valA/{s}_{img}'
    shutil.copy(src, dst)

In [ ]:
train, val, test = [], [], []
for s in hamamatsu_slides:
  slide_dir = os.path.join(root_dir, s)
  imgs = [os.path.join(root_dir, s, i) for i in os.listdir(slide_dir)]
  random.shuffle(imgs)
  subtrain, subtest = train_test_split(imgs, test_size=test_size)
  if enable_val:
    subval, subtest = train_test_split(subtest, test_size=val_size)
    val.extend(subval)
  train.extend(subtrain)
  test.extend(subtest)

In [ ]:
os.makedirs('data/trainB')
os.makedirs('data/testB')
if enable_val:
  os.makedirs('data/valB')

In [ ]:
for src in train:
  s = src.split('/')[-2]
  img = src.split('/')[-1]
  dst = f'data/trainB/{s}_{img}'
  shutil.copy(src, dst)

for src in test:
  s = src.split('/')[-2]
  img = src.split('/')[-1]
  dst = f'data/testB/{s}_{img}'
  shutil.copy(src, dst)

if enable_val:
  for src in val:
    s = src.split('/')[-2]
    img = src.split('/')[-1]
    dst = f'data/valB/{s}_{img}'
    shutil.copy(src, dst)

train

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
os.makedirs('checkpoints/experiment_name/log')

In [ ]:
!python train.py --config config/train_config.json

test

In [ ]:
!python test.py --config config/test_config.json